In [1]:
#required for colab
#path = '/content/drive/MyDrive/PGD-DL-B3n4/09-10/2.1 Yolo-v3'
#import os
#os.chdir(path)

In [2]:
#required only once if files are not there
#!wget "https://pjreddie.com/media/files/yolov3.weights"
#!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
#!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  7 23:02:42 2022

@author: tariq
"""

import cv2 as cv
import numpy as np

width, height = 416, 416
confThreshold =0.5
nmsThreshold= 0.2

#### LOAD MODEL
## Coco Names
classesFile = "coco.names"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('n').split('n')
print(classNames)

FileNotFoundError: ignored

In [ ]:
import cv2 as cv

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolov3.cfg"
modelWeights = "yolov3.weights"

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


In [ ]:
def getOutputsNames(net):
   # Get the names of all the layers in the network
   layersNames = net.getLayerNames()
   # Get the names of the output layers, i.e. the layers with unconnected outputs
   return [layersNames[i - 1] if isinstance(i, int) else layersNames[i - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
#video = '/content/drive/MyDrive/PGD-DL-B3n4/09-10/Data/Final_Fifa.mp4'  # for colab
video = '/home/tjamil/NED_PGD/PGD-DL-B3n4/09-10/Data/Final_Fifa.mp4'

In [ ]:
import matplotlib.pyplot as plt

cap = cv.VideoCapture(video)
print(width, height)

while(cap.isOpened()):
    ret, frame = cap.read()
    #print(ret)
    if not ret:
        break

    frame = cv.resize(frame, (width, height))


    # Create a 4D blob from a frame.
    blob = cv.dnn.blobFromImage(frame, 1/255, (width, height), (0, 0, 0), True, crop=True)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output from the output layers
    outs = net.forward(getOutputsNames(net))

    # Remove the bounding boxes with low confidence
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv.imshow("Video", frame)
    key = cv.waitKey(1)
    if key == 27:
        break

cap.release()
cv.destroyAllWindows()